# Gnod case study: Top 100 songs recommender

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
from time import sleep

# Lab 6.1: Scraping top 100 chart from popvortex.com and converting to dataframe

In [39]:
url = "https://www.popvortex.com/music/charts/top-100-songs.php"

In [40]:
response = requests.get(url)
response.status_code

200

In [41]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
print(soup.prettify())

In [ ]:
# Selector script that we want to parse

#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > p > cite

#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > ul > li:nth-child(1) > a

In [ ]:
soup.select("cite.title")
soup.select('em.artist')
soup.select('p.chart-position')

In [26]:
soup.select('cite.title')[0].get_text()

'Unholy'

In [39]:
soup.select('em.artist')[0].get_text()


'Sam Smith & Kim Petras'

In [103]:
soup.select('p.chart-position')[0].get_text()

'1'

In [153]:
soup.select('#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > ul > li:nth-child(1) > a')[0].get_text()

'Pop'

In [166]:
soup.select('#div.chart-content.col-xs-12.col-sm-8 > ul')

[]

In [ ]:
soup.select('li:a')[0].get_text()

In [ ]:
# Working on getting the Genre from popvortex but its more difficult so sticking with the three for now.

# genre = []

# for g in soup.select("li"):
#     link = soup.get("href")
#     print(link)
#     if link is not None:
#         if (("/music" not in link) & 
#             ("/charts" not in link)):
#             genre.append(soup["href"])

In [42]:
#initialize empty lists
title = []
artist = []
rank = []
genre = []

# Number of iterations
num_iter = len(soup.select("cite.title"))

# Setting up what to iterate for each empty list
t_list = soup.select("cite.title")
a_list = soup.select("em.artist")
chart_rank = soup.select('p.chart-position')
gen = soup.select('#chart-position-1 > div.chart-content.col-xs-12.col-sm-8 > ul > li:nth-child(1) > a')

# iterate through the result set and retrive all the data
for i in range(num_iter):
    title.append(t_list[i].get_text())
    artist.append(a_list[i].get_text())
    rank.append(chart_rank[i].get_text())
    genre.append(gen[0])

# Checking results
# print(title)
# print(artist)
# print(rank)
# print(genre)

In [43]:
# Converting our lists into a dataframe

top_100 = pd.DataFrame({"rank":rank,
                        "title":title,
                       "artist":artist,
                       "genre":genre
                      })

In [44]:
# We now see 100 rows with rank, title and artist information (note: not indexed to rank)

# Note: We are still working on making the genre column correct...

top_100

,rank,title,artist,genre
0,1,Unholy,Sam Smith & Kim Petras,[Pop]
1,2,Eagle (feat. KB),Transformation Worship,[Pop]
2,3,Everywhere,Fleetwood Mac,[Pop]
3,4,I'm Good (Blue),David Guetta & Bebe Rexha,[Pop]
4,5,wait in the truck,HARDY & Lainey Wilson,[Pop]
...,...,...,...,...
95,96,Boulevard of Broken Dreams (feat. John Gallagh...,Green Day,[Pop]
96,97,Top Of The World,Shawn Mendes,[Pop]
97,98,Ghostbusters,Ray Parker Jr.,[Pop]
98,99,Perfectly Loved (feat. TobyMac),Rachael Lampa,[Pop]


In [45]:
# Recommendations (for a random song) based on user input

print('Welcome to Gnod song recommender!')
print('Enter the name of your favorite song to recieve a recommendation:') 

str(input())

rand_idx = random.randrange(len(top_100['title']))
random_rec = title[rand_idx]

print('Based on your input, we recommend: ' + random_rec)

Welcome to Gnod song recommender!
Enter the name of your favorite song to recieve a recommendation:
Based on your input, we recommend: Bedroom Singer


# Lab 6.2: Scraping multiple sheets

In [174]:
url2 = "https://www.popvortex.com/music/france/top-songs.php"

In [175]:
response2 = requests.get(url2)

soup2 = BeautifulSoup(response2.content, "html.parser")

In [182]:
# Expanding the sample of songs

#initialize empty lists
title2 = []
artist2 = []

# Number of iterations
num_iter2 = len(soup2.select("cite.title"))

# Setting up what to iterate for each empty list
t_list2 = soup2.select("cite.title")
a_list2 = soup2.select("em.artist")

# iterate through the result set and retrive all the data
for i in range(num_iter2):
    title2.append(t_list2[i].get_text())
    artist2.append(a_list2[i].get_text())

In [183]:
top_100_france = pd.DataFrame({"title":title,
                       "artist":artist,
                      })

In [184]:
songs = top_100.copy()

In [188]:
songs = songs.drop(['rank', 'genre'], axis=1)

In [192]:
# Concatenating the top 100 list from the US with that of France

songs_list = pd.concat([songs, top_100_france], axis=0).reset_index(drop=True)

songs_list

,title,artist
0,Unholy,Sam Smith & Kim Petras
1,Eagle (feat. KB),Transformation Worship
2,Everywhere,Fleetwood Mac
3,I'm Good (Blue),David Guetta & Bebe Rexha
4,wait in the truck,HARDY & Lainey Wilson
...,...,...
195,We Don't Talk About Bruno,"Carolina Gaitán - La Gaita, Mauro Castillo, Ad..."
196,Me Porto Bonito,Bad Bunny & Chencho Corleone
197,Whiskey Glasses,Morgan Wallen
198,Don't You (Forget About Me),Simple Minds


In [46]:
# Removing duplicates (looks like 17 rows removed)

songs_list.drop_duplicates()

,title,artist
0,Calm Down,Rema
1,Call on me (feat. Ed Sheeran),Vianney
2,I'm Good (Blue),David Guetta & Bebe Rexha
3,As It Was,Harry Styles
4,Don't You Worry,"Black Eyed Peas, Shakira & David Guetta"
...,...,...
195,Boulevard of Broken Dreams (feat. John Gallagh...,Green Day
196,Top Of The World,Shawn Mendes
197,Ghostbusters,Ray Parker Jr.
198,Perfectly Loved (feat. TobyMac),Rachael Lampa


# Same operation as above, but using a function

In [4]:
def scraper(url):

    response3 = requests.get(url)
    soup3 = BeautifulSoup(response3.content, "html.parser")

    titles = []
    artists = []

    num_iters = len(soup3.select("cite.title"))

    t_lists = soup3.select("cite.title")
    a_lists = soup3.select("em.artist")

    for i in range(num_iters):
        titles.append(t_lists[i].get_text())
        artists.append(a_lists[i].get_text())
    
    test = pd.DataFrame({"title":titles,
                       "artist":artists})
    return test

In [8]:
urls1 = 'https://www.popvortex.com/music/france/top-songs.php'
urls2 = 'https://www.popvortex.com/music/charts/top-100-songs.php'

france_100 = scraper(urls1)
us_100 = scraper(urls2)

In [9]:
songs_list = pd.concat([france_100, us_100], axis=0).reset_index(drop=True)

songs_list

,title,artist
0,Calm Down,Rema
1,Call on me (feat. Ed Sheeran),Vianney
2,I'm Good (Blue),David Guetta & Bebe Rexha
3,As It Was,Harry Styles
4,Don't You Worry,"Black Eyed Peas, Shakira & David Guetta"
...,...,...
195,Boulevard of Broken Dreams (feat. John Gallagh...,Green Day
196,Top Of The World,Shawn Mendes
197,Ghostbusters,Ray Parker Jr.
198,Perfectly Loved (feat. TobyMac),Rachael Lampa


In [10]:
songs_list.drop_duplicates()

,title,artist
0,Calm Down,Rema
1,Call on me (feat. Ed Sheeran),Vianney
2,I'm Good (Blue),David Guetta & Bebe Rexha
3,As It Was,Harry Styles
4,Don't You Worry,"Black Eyed Peas, Shakira & David Guetta"
...,...,...
195,Boulevard of Broken Dreams (feat. John Gallagh...,Green Day
196,Top Of The World,Shawn Mendes
197,Ghostbusters,Ray Parker Jr.
198,Perfectly Loved (feat. TobyMac),Rachael Lampa


# Practicing scraping with another website - CoinMarketCap

In [11]:
url_cmc = "https://coinmarketcap.com/"

In [12]:
response_cmc = requests.get(url_cmc)

soup_cmc = BeautifulSoup(response_cmc.content, "html.parser")

In [14]:
soup_cmc.select('p > span.sc-1ow4cwt-1.ieFnWP')[0].get_text()

'$367,990,849,787'

In [28]:
soup_cmc.select('div > a > div > div > p')[0].get_text()

'Bitcoin'

In [34]:
coin = []
mcap = []

num_iters = len(soup_cmc.select('p > span.sc-1ow4cwt-1.ieFnWP'))

coin_list = soup_cmc.select('div > a > div > div > p')
mcap_list = soup_cmc.select('p > span.sc-1ow4cwt-1.ieFnWP')

for i in range(num_iters):
    coin.append(coin_list[i].get_text())
    mcap.append(mcap_list[i].get_text())
    
cmc_data = pd.DataFrame({"coin":coin,
                       "marketcap":mcap})

In [35]:
cmc_data

,coin,marketcap
0,Bitcoin,"$367,990,849,787"
1,Ethereum,"$159,984,368,270"
2,Tether,"$68,374,239,646"
3,USD Coin,"$46,060,977,592"
4,BNB,"$44,156,539,145"
5,XRP,"$25,849,392,338"
6,Binance USD,"$21,614,987,817"
7,Cardano,"$14,192,698,841"
8,Solana,"$11,620,521,084"
9,Dogecoin,"$8,013,357,970"
